In [1]:
import sparknlp
sparknlp.start()

from sparknlp.base import *
from sparknlp.annotator import * 
from sparknlp.annotator import GraphExtraction

from pyspark.ml import Pipeline
from sparknlp.pretrained import PretrainedPipeline
from pyspark.sql import SparkSession

from graphframes import *

In [2]:
# !pip install graphframes

In [3]:
spark = SparkSession.builder.appName('Spark-NLP').getOrCreate()

In [4]:
spark

In [5]:
ner_pipeline = PretrainedPipeline("recognize_entities_dl", lang="en")

recognize_entities_dl download started this may take some time.
Approx size to download 160.1 MB
[ | ]recognize_entities_dl download started this may take some time.
Approximate size to download 160.1 MB
Download done! Loading the resource.
[ / ]

[ — ]

[ \ ]

[ | ]

21/12/02 03:16:02 WARN org.apache.hadoop.util.concurrent.ExecutorHelper: Thread (Thread[GetFileInfo #1,5,main]) interrupted: 
java.lang.InterruptedException
	at com.google.common.util.concurrent.AbstractFuture.get(AbstractFuture.java:510)
	at com.google.common.util.concurrent.FluentFuture$TrustedFuture.get(FluentFuture.java:88)
	at org.apache.hadoop.util.concurrent.ExecutorHelper.logThrowableFromAfterExecute(ExecutorHelper.java:48)
	at org.apache.hadoop.util.concurrent.HadoopThreadPoolExecutor.afterExecute(HadoopThreadPoolExecutor.java:90)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1157)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)


...

21/12/02 03:16:12 WARN org.apache.hadoop.util.concurrent.ExecutorHelper: Thread (Thread[GetFileInfo #1,5,main]) interrupted: 
java.lang.InterruptedException
	at com.google.common.util.concurrent.AbstractFuture.get(AbstractFuture.java:510)
	at com.google.common.util.concurrent.FluentFuture$TrustedFuture.get(FluentFuture.java:88)
	at org.apache.hadoop.util.concurrent.ExecutorHelper.logThrowableFromAfterExecute(ExecutorHelper.java:48)
	at org.apache.hadoop.util.concurrent.HadoopThreadPoolExecutor.afterExecute(HadoopThreadPoolExecutor.java:90)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1157)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)


[ / ]

[ \ ]

2021-12-02 03:16:19.723223: I external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-02 03:16:19.887278: W external/org_tensorflow/tensorflow/core/common_runtime/colocation_graph.cc:1139] Failed to place the graph without changing the devices of some resources. Some of the operations (that had to be colocated with resource generating operations) are not supported on the resources' devices. Current candidate devices are [
  /job:localhost/replica:0/task:0/device:CPU:0].
See below for details of this colocation group:
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=-1 requested_device_name_='/device:GPU:0' assigned_de

[OK!]


In [5]:
# onto_ner_pipeline = PretrainedPipeline("onto_recognize_entities_sm", lang="en")

In [10]:
sentences = [
  ['Apple is looking at buying U.K. startup for $1 billion. Google might make their own offer.'],
  ['Amazon is the largest forest in Brazil. Amazon is the largest retailer in the world']
]

data = spark.createDataFrame(sentences).toDF("text")
# data.show()

In [6]:
df = spark.read.option("multiline","true").option("header", "true").option("ignoreTrailingWhiteSpace","true").\
option("ignoreLeadingWhiteSpace", "true").option("inferSchema", "true")\
.csv("gs://bigdata_project_hksv/tweetDataSub/*.csv")
# .csv("gs://bigdata_project_hksv/tweetDataSub/2020-01_20211022-17.30.23.csv")

In [7]:
df1 = df.dropna(thresh=1, subset=('tweet'))
df1 = df1.filter(df1.tlang == "en")

In [8]:
from pyspark.sql.functions import regexp_replace, col
df2 = df1.withColumn('tweet1',regexp_replace(col('tweet'), '@', ''))
df2 = df2.withColumn('tweet2',regexp_replace(col('tweet1'), '#', ''))
df2 = df2.withColumn('tweet2',regexp_replace(col('tweet2'), 'RT', ''))
df2 = df2.withColumn('tweet2',regexp_replace(col('tweet2'), ':', ''))
df2 = df2.withColumn('tweet2',regexp_replace(col('tweet2'), 'http\S+', ''))

In [9]:
data = df2.withColumn('text',col('tweet2'))

In [10]:
data.show(3, False)

+-------------------+----------+--------+-----+--------------+----------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------

In [11]:
annotation = ner_pipeline.transform(data)
# annotation.show()

In [12]:
annotation.select('entities.result', 'entities.metadata').show(truncate=False)

21/12/02 03:17:09 WARN org.apache.spark.sql.catalyst.util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|result                                                                                    |metadata                                                                                                                                                                                                                     |
+------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[Trump, Mission Accomplished, Walter Reed]            

In [13]:
documentAssembler     = DocumentAssembler().setInputCol("text").setOutputCol("document")
tokenizer             = Tokenizer().setInputCols(["document"]).setOutputCol("token")
normalizer            = Normalizer().setInputCols(["token"]).setOutputCol("normalized").setCleanupPatterns(["[^\w\d\s]"])
lemmatizer            = LemmatizerModel.pretrained().setInputCols(['normalized']).setOutputCol('lemmatized')
stopwords_cleaner     = StopWordsCleaner().setInputCols("lemmatized").setOutputCol("cleanTokens").setCaseSensitive(False)\
      .setStopWords(['i','me','my','myself','we','our','ours','ourselves','you','your','yours','yourself',\
                      'yourselves','he','him','his','himself','she','her','hers','herself','it','its','itself',\
                      'they','them','their','theirs','themselves','what','which','whom','this','that','these',\
                      'those','is','are','was','were','be','been','being','have','has','had','having','do',\
                      'does','did','doing','a','an','the','and','but','if','or','because','as','until','while',\
                      'of','at','by','for','with','about','against','between','into','through','during','before',\
                      'after','above','below','to','from','in','out','on','off','over','under','again','further',\
                      'then','once','here','there','when','where','why','how','all','any','both','each','few',\
                      'more','most','other','some','such','nor','only','own','same','so','than','too','very','s',\
                      't','can','will','just','don','should','now',"i'll","you'll","he'll","she'll","we'll",\
                      "they'll","i'd","you'd","he'd","she'd","we'd","they'd","i'm","you're","he's","she's","it's",\
                      "we're","they're","i've","we've","you've","they've","isn't","aren't","wasn't","weren't",\
                      "haven't","hasn't","hadn't","don't","doesn't","didn't","won't","wouldn't","shan't",\
                      "shouldn't","mustn't","can't","couldn't",'cannot','could',"here's","how's","let's",'ought',\
                      "that's","there's","what's","when's","where's","why's",'would','no','not','get','via','amp'])
posTagger             = PerceptronModel.pretrained().setInputCols(["cleanTokens", "document"]).setOutputCol("pos")
embeds                = WordEmbeddingsModel.pretrained().setInputCols(["cleanTokens", "document"]).setOutputCol("embeddings")
nerCrf                = NerCrfModel.pretrained().setInputCols(["document", "cleanTokens","pos", "embeddings"]).setOutputCol("ner")


# ner_tagger = MedicalNerModel()\
#     .pretrained("ner_posology", "en", "clinical/models")\
#     .setInputCols(["document", "cleanTokens", "embeddings"])\
#     .setOutputCol("ner_tags")    

ner_chunker = NerConverter()\
    .setInputCols(["document", "cleanTokens", "ner"])\
    .setOutputCol("ner_chunks")

dependency_parser = DependencyParserModel()\
    .pretrained("dependency_conllu", "en")\
    .setInputCols(["document", "pos", "cleanTokens"])\
    .setOutputCol("dependencies")

# reModel = RelationExtractionModel()\
#     .pretrained("posology_re")\
#     .setInputCols(["embeddings", "pos", "ner_chunks", "dependencies"])\
#     .setOutputCol("relations")\
#     .setMaxSyntacticDistance(4)

graph1 = GraphExtraction().setInputCols(["document", "cleanTokens", "ner"]).setOutputCol("graph").\
setRelationshipTypes(["prefer-LOC"]).setMergeEntities(True)


lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[ | ]lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
Download done! Loading the resource.
[OK!]
pos_anc download started this may take some time.
Approximate size to download 3.9 MB
[ | ]pos_anc download started this may take some time.
Approximate size to download 3.9 MB
Download done! Loading the resource.


[ / ]

[OK!]
glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[ | ]glove_100d download started this may take some time.
Approximate size to download 145.3 MB
Download done! Loading the resource.
[OK!]
ner_crf download started this may take some time.
Approximate size to download 10.2 MB
[ | ]ner_crf download started this may take some time.
Approximate size to download 10.2 MB
Download done! Loading the resource.


[ — ]

[OK!]
dependency_conllu download started this may take some time.
Approximate size to download 16.7 MB
[ | ]dependency_conllu download started this may take some time.
Approximate size to download 16.7 MB
Download done! Loading the resource.


[ | ]

[OK!]


In [15]:
pipeline = Pipeline(stages=[documentAssembler, tokenizer, normalizer, lemmatizer, stopwords_cleaner, \
                            posTagger, embeds, nerCrf, dependency_parser])
result = pipeline.fit(data).transform(data)
result.select("ner","dependencies").show(truncate = False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [16]:
# graph1.fit(result).transform(result)
result.select("ner.result").show(truncate = False)

+-------------------------------------------------------------------------------------------------------------+
|result                                                                                                       |
+-------------------------------------------------------------------------------------------------------------+
|[I-PER, O, O, O, O, O, O, O, O, O, O, O, I-ORG, I-ORG, O, O, O, O, O, O, I-PER, I-PER, O, O, O]              |
|[I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG]                  |
|[O, O, I-ORG, I-ORG, I-ORG, I-ORG, O, O, O, O, I-ORG, I-ORG, O, O, O]                                        |
|[I-ORG, O, O, O, O, O, O, O]                                                                                 |
|[I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG]                                                            |
|[O, O]                                                                                                 

In [17]:
# result.show(1, False)
# sparknlp.version()

In [18]:
annotation.select('entities.result').show(10, truncate=False)

+------------------------------------------------------------------------------------------+
|result                                                                                    |
+------------------------------------------------------------------------------------------+
|[Trump, Mission Accomplished, Walter Reed]                                                |
|[Secret Service Agents, Walter Reed, Perez Hilton SmartNews]                              |
|[Regence, Oregon Health Authority]                                                        |
|[NYC, COVID-19]                                                                           |
|[Lewis &amp, Bunting Denied!, Darts WGP20]                                                |
|[]                                                                                        |
|[Donald Trump, Coronavirus, Don't, Trump Trump2020NowMoreThanEver 
TRUMP2020ToSaveAmerica]|
|[Amazing]                                                            

In [19]:
from pyspark.sql import functions as F
preds = result.select(F.explode(F.arrays_zip('cleanTokens.result','ner.result')).alias("cols"))\
.select(F.col('cols.0').alias("cleanTokens"), F.col('cols.1').alias("ner"))

In [20]:
# preds.filter(~col("ner").contains("O")).show(10)
preds.show(10)

+-----------+-----+
|cleanTokens|  ner|
+-----------+-----+
|      Trump|I-PER|
|       look|    O|
|       like|    O|
|        man|    O|
|        try|    O|
|    contain|    O|
|    disdain|    O|
|   shouldve|    O|
|       feel|    O|
|      great|    O|
+-----------+-----+
only showing top 10 rows



In [21]:
preds1 = preds.filter(~col("ner").contains("O"))

In [22]:
ner_count = preds.groupBy("ner").count().sort(col("count").desc())
ner_count_wo_O = preds1.groupBy("ner").count().sort(col("count").desc())

In [23]:
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

21/12/02 03:18:35 WARN org.apache.spark.sql.internal.SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.


In [24]:
# ner_count.show()

In [ ]:
pd_ner_count = ner_count.toPandas()

In [ ]:
# pd_ner_count_wo_O = ner_count_wo_O.toPandas()

In [ ]:
preds_new = result.withColumn("cleanTokens", F.explode('cleanTokens.result')).\
withColumn("ner", F.explode('ner.result'))

# result.select(F.explode(F.arrays_zip('cleanTokens.result','ner.result')).alias("cols"))\
# .select(F.col('cols.0').alias("cleanTokens"), F.col('cols.1').alias("ner"), F.col('cols.2').alias("dep_"))

In [26]:
# preds1.show()

In [24]:
graph_df = result.select(col('cleanTokens.result').alias("cleanTokens"),col('pos.result').alias("pos"),\
                         col('ner.result').alias("ner"),col('dependencies.result').alias("dependencies"),\
                        col("tcountry_place").alias("country"))
# .\
# select(F.col('cols.0').alias("cleanTokens"), F.col('cols.1').alias("pos"), F.col('cols.2').alias("ner"))

graph_df.show(10, False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+
|cleanTokens                                                                                                                                                                                                                         

In [25]:
# graph_df_ent = graph_df.select("ner", )
graph_df.groupBy("country").count().show()

+-------+------+
|country| count|
+-------+------+
|     CN|     1|
|     AU|     1|
|     CA|     2|
|     GB|     2|
|     US|     7|
|     PH|     3|
|     CY|     1|
|     CO|     1|
|   NULL|125272|
+-------+------+



In [62]:
graph_df_rel = graph_df.filter(col("pos").isin("VBZ","VBN")).filter(~col("token").isin("is","has"))
graph_df_rel.show(50)

+--------------+---+---+
|         token|pos|ner|
+--------------+---+---+
|      declared|VBN|  O|
|          been|VBN|  O|
|     developed|VBN|  O|
|          made|VBN|  O|
|      infected|VBN|  O|
|         means|VBZ|  O|
|        banned|VBN|  O|
|        locked|VBN|  O|
|      declared|VBN|  O|
|      provides|VBZ|  O|
|    engineered|VBN|  O|
|      released|VBN|  O|
|      declares|VBZ|  O|
|       travels|VBZ|  O|
|      declares|VBZ|  O|
|     evacuated|VBN|  O|
|        placed|VBN|  O|
|         hurts|VBZ|  O|
|          seen|VBN|  O|
|      declares|VBZ|  O|
|        sounds|VBZ|  O|
|       reaches|VBZ|  O|
|       written|VBN|  O|
|      declares|VBZ|  O|
|‘unprecedented|VBN|  O|
|      infected|VBN|  O|
|          made|VBN|  O|
|      declares|VBZ|  O|
|         knows|VBZ|  O|
|     concerned|VBN|  O|
|          Does|VBZ|  O|
|      declares|VBZ|  O|
|     confirmed|VBN|  O|
|     confirmed|VBN|  O|
|      infected|VBN|  O|
|   quarantined|VBN|  O|
|         means|VBZ|  O|


In [9]:
# !echo ${SPARK_HOME}/jars

In [10]:
# !pyspark --version
# !which pyspark

In [11]:
# !wget /usr/lib/spark/jars/ "https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.1-s_2.12/graphframes-0.8.2-spark3.1-s_2.12.jar"

In [12]:
# !ls

In [13]:
# !cp graphframes-0.8.2-spark3.1-s_2.12.jar /usr/lib/spark/jars/
# !cp graphframes-0.8.2-spark3.1-s_2.12.jar ./root/.ivy2/jars/